In [1]:
import sys
import os
import glob
import json

# Import local packages
sys.path.append('../src/aerospace_chatbot')
import admin
import data_processing

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(), override=True)

2024-11-02 13:16:52.199 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-11-02 13:16:52.200 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


True

# Setup

In [2]:
# Make a function to split chunking from upserting
def create_upsert(index_type,index_name,query_model,rag_type,chunker,summary_llm):
    # Set index names for special databases
    if rag_type == 'Parent-Child':
        index_name = index_name + '-parent-child'
    if rag_type == 'Summary':
        index_name = index_name + '-' + summary_llm.model_name.replace('/', '-').replace(' ','-').lower() + '-summary' 

    try:
        vectorstore = data_processing.initialize_database(index_type, 
                                            index_name, 
                                            query_model,
                                            rag_type=rag_type,
                                            clear=True, 
                                            local_db_path=os.getenv('LOCAL_DB_PATH'),
                                            init_ragatouille=True,
                                            show_progress=False)
        print(f"Database {index_name} created.")
        vectorstore, _ = data_processing.upsert_docs(index_type, 
                                        index_name,
                                        vectorstore,
                                        chunker,
                                        batch_size=400,
                                        show_progress=False,
                                        local_db_path=os.getenv('LOCAL_DB_PATH'))
        print(f"Database {index_name} upserted chunks.")
    except Exception as e:  # If there is an error, be sure to delete the database
        data_processing.delete_index(index_type, 
                                    index_name,
                                    rag_type,
                                    local_db_path=os.getenv('LOCAL_DB_PATH'))
        print(f"Database deleted: {index_name}")
        print(f"Error: {e}")

## Secrets, Models, Docs, Params

In [3]:
# Set secrets
secrets={}
sb={}

secrets['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
secrets['VOYAGE_API_KEY'] = os.getenv('VOYAGE_API_KEY')
secrets['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
secrets['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
# Read setup data, assign models
json_file_path = "databases_minimal.json"
with open(json_file_path, "r") as json_file:
    setup_data = json.load(json_file)

sb={}
query_params=setup_data['query_models']
query_models=[]
for model in query_params:
    for key in model:
        sb[key] = model[key]
    query_models.append(admin.get_query_model(sb, secrets))

llm_params=setup_data['llms']
llms=[]
for model in llm_params:
    for key in model:
        sb[key] = model[key]
    llms.append(admin.set_llm(sb, secrets))

chunk_params=setup_data['chunk_params']

In [6]:
# Get docs
data_folder='../data/AMS'
docs= glob.glob(os.path.join(data_folder,'*.pdf'))   # Only get the PDFs in the directory

# Make Databases

In [7]:
index_tag='ams'

## OpenAI text-embedding-3-large, standard, 2 page merge, no chunk

In [11]:
rag_type='Standard'
summary_llm=llms[0] # gpt-4o

i_chunk=0   # 2 page merge, no chunk
i_run=0     # OpenAI text-embedding-3-large

In [14]:
chunker=data_processing.chunk_docs(docs,
            rag_type=rag_type,
            n_merge_pages=chunk_params[i_chunk]['n_merge_pages'],
            chunk_method=chunk_params[i_chunk]['chunk_method'],
            chunk_size=chunk_params[i_chunk]['chunk_size'],
            llm=summary_llm,
            show_progress=False)

print(f"Created {len(chunker['chunks'])} chunks from {len(chunker['pages'])} pages.")

Created 340 chunks from 340 pages.


In [15]:
# Create and upsert database
print(f"Creating and uploading database with these params: {query_params[i_run]}")

index_appendix=str(chunk_params[i_chunk]['n_merge_pages'])+'merge'+'-'+str(chunk_params[i_chunk]['chunk_size'])
index_name = (query_params[i_run]['embedding_name'].replace('/', '-').replace(' ', '-') 
                + '-' + index_tag + '-' + index_appendix).lower()

create_upsert(query_params[i_run]['index_type'],
                index_name,
                query_models[i_run],
                rag_type,
                chunker,
                summary_llm)

Creating and uploading database with these params: {'id': '1', 'index_type': 'ChromaDB', 'query_model': 'OpenAI', 'embedding_name': 'text-embedding-3-large'}
Database text-embedding-3-large-ams-2merge-0 created.
Database text-embedding-3-large-ams-2merge-0 upserted chunks.
